## Feature engineering

In [ ]:
CPGF_df.info()

In [ ]:
### ID Portador
CPGF_df["ID_portador"] = CPGF_df["CPF PORTADOR"] + CPGF_df["NOME PORTADOR"]
### Fim de semana -- alterado de 99 para 0 os valores sigilosos
CPGF_df["fim_semana"] = np.where(CPGF_df['DADO_SIGILOSO'] == 1, 0, CPGF_df['DATA TRANSAÇÃO'].dt.day_of_year.isin([5, 6]).astype(int))

In [ ]:
CPGF_df.head()

In [ ]:
CPGF_df['DADO_SIGILOSO'].value_counts()

In [ ]:
CPGF_df["fim_semana"].value_counts()

In [ ]:
CPGF_df['NOME ÓRGÃO SUPERIOR'].value_counts().cumsum()

In [ ]:
### Agregado por órgão superior

summary_orgao_superior = (
    CPGF_df.groupby("NOME ÓRGÃO SUPERIOR")["VALOR TRANSAÇÃO"]
      .agg([
          ("mean", "mean"),
      ])
      .sort_values("mean", ascending=False)
).reset_index()

summary_orgao_superior


In [ ]:
summary_orgao_superior.plot(kind='bar', figsize=(12,6), legend = False)
plt.title("Média de Valor por Órgão Superior")
plt.ylabel("Média do Valor")
plt.xlabel("Órgão Superior")
plt.tight_layout()
plt.show();

In [ ]:
### Agregado por órgão

summary_orgao = (
    CPGF_df.groupby("NOME ÓRGÃO")["VALOR TRANSAÇÃO"]
      .agg([
          ("mean", "mean"),
      ])
      .sort_values("mean", ascending=False)
).reset_index()

summary_orgao

In [ ]:
top_30_orgao = summary_orgao.head(30)
top_30_orgao.plot(kind='bar', figsize = (12,6), legend=False)
plt.title("Média de Valor por Órgão")
plt.ylabel("Média do Valor")
plt.xlabel("Órgão")
plt.tight_layout()
plt.show();

In [ ]:
### Agregado por unidade gestora

summary_unidade_gestora = (
    CPGF_df.groupby("NOME UNIDADE GESTORA")["VALOR TRANSAÇÃO"]
      .agg([
          ("mean", "mean"),
      ])
      .sort_values("mean", ascending=False)
).reset_index()

summary_unidade_gestora

In [ ]:
top_30_unidade_gestora = summary_unidade_gestora.head(30)
top_30_unidade_gestora.plot(kind='bar', figsize = (12,6), legend=False)
plt.title("Média de Valor por Unidade Gestora")
plt.ylabel("Média do Valor")
plt.xlabel("Unidade Gestora")
plt.tight_layout()
plt.show();

In [ ]:
### Agregado por Estado

summary_estado = (
    CPGF_df.groupby("ESTADO_ESTIMADO")["VALOR TRANSAÇÃO"]
      .agg([
          ("mean", "mean"),
      ])
      .sort_values("mean", ascending=False)
).reset_index()

summary_estado

In [ ]:
PR_df = CPGF_df[CPGF_df['ESTADO_ESTIMADO'] == 'PR'].sort_values('VALOR TRANSAÇÃO', ascending=False)
PR_df.head(2)

In [ ]:
summary_PR = (
    PR_df.groupby("NOME UNIDADE GESTORA")["VALOR TRANSAÇÃO"]
      .agg([
          ("mean", "mean"),
          ("sum", "sum"),
      ])
      .sort_values("sum", ascending=False)
)

summary_PR

In [ ]:
PR_df[PR_df['NOME UNIDADE GESTORA'] == 'SUPERINT REG DO TRAB E EMPREGO/PR'].sort_values('VALOR TRANSAÇÃO', ascending=False)

In [ ]:
summary_estado.plot(kind='bar', figsize = (12,6), legend=False)
plt.title("Média de Valor por Estado")
plt.ylabel("Média do Valor")
plt.xlabel("Estado")
plt.tight_layout()
plt.show();

In [ ]:
def feature_engineering(df):
    # ID_portador column
    df["ID_portador"] = df["CPF PORTADOR"] + df["NOME PORTADOR"]

    # Weekend column (including "SIGILOSO" rule)
    df["fim_semana"] = np.where(
        df['DADO_SIGILOSO'] == 1,
        99,
        df['DATA TRANSAÇÃO'].dt.day_of_year.isin([5, 6]).astype(int)
    )

    # Transform value into log
    df['LOG_VALOR'] = np.log1p(df['VALOR TRANSAÇÃO'])

    return df

In [ ]:
## Frequência das categorias
cols_categoria = ['NOME ÓRGÃO', 'ESTADO_ESTIMADO', 'NOME FAVORECIDO']

for col in cols_categoria:
    freq_map = df_clean[col].value_counts(normalize=True)
    df_clean[f'FREQ_{col}'] = df_clean[col].map(freq_map)

In [ ]:
# Média por Órgão no MÊS
df_clean['MEDIA_VALOR_ORGAO_MES'] = df_clean.groupby(['NOME ÓRGÃO', 'ANO EXTRATO', 'MÊS EXTRATO'])['VALOR TRANSAÇÃO'].transform('mean')

In [ ]:
# Razão do valor da transação pela média do órgão no período
df_clean['RATIO_MES'] = df_clean['VALOR TRANSAÇÃO'] / df_clean['MEDIA_VALOR_ORGAO_MES']